<a href="https://colab.research.google.com/github/j-buss/wi-dpi-analysis/blob/dev-adj-inflation/pipeline/python/process/adjInflation/CPI_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery
%matplotlib inline
plt.style.use('bmh')

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

In [0]:
project_id='wi-dpi-010'
dataset_name='2016'

In [0]:
bq_client = bigquery.Client(project=project_id)

In [0]:
sql='''
  SELECT
    id_nbr,
    salary,
    benefits
  FROM
    `{}.{}`
  LIMIT
    1000
'''.format(dataset_name,'2016_BASE')

In [0]:
temp_df = pd.read_gbq(sql, project_id=project_id)

In [30]:
!pip install cpi

     |████████████████████████████████| 25.1MB 92kB/s 
     |████████████████████████████████| 235kB 62.0MB/s 
ERROR: botocore 1.13.14 has requirement python-dateutil<2.8.1,>=2.1; python_version >= "2.7", but you'll have python-dateutil 2.8.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: python-dateutil 2.6.1
    Uninstalling python-dateutil-2.6.1:
      Successfully uninstalled python-dateutil-2.6.1


In [31]:
import cpi

/usr/local/lib/python3.6/dist-packages/cpi/__init__.py:46: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  warnings.warn(StaleDataWarning())


In [0]:
cpi.update()

In [0]:
temp_df['salary_adj'] = temp_df.apply(lambda x: cpi.inflate(x.salary, int(dataset_name)), axis=1)
temp_df['benefits_adj'] = temp_df.apply(lambda x: cpi.inflate(x.benefits, int(dataset_name)), axis=1)

In [40]:
temp_df

,id_nbr,salary,benefits,salary_adj,benefits_adj
0,688913,175320,42352,183428.313508,44310.722871
1,646275,121663,27673,127289.749636,28952.838921
2,627194,181549,68550,189945.396355,71720.345032
3,674556,100000,18723,104624.865108,19588.913494
4,602929,97345,22075,101847.074939,23095.938973
...,...,...,...,...,...
995,665804,100000,32714,104624.865108,34226.978371
996,677142,103525,33309,108312.891603,34849.496319
997,591685,122000,41559,127642.335432,43481.047690
998,604048,120691,37424,126272.795948,39154.809518
